# Test

In [1]:
#@markdown #Installation
#@markdown *Run this cell to install MVSep-MDX23*
print('Installing... This will take 1 minute...')
#%cd /content
#from google.colab import drive
#drive.mount('/content/drive')
import os
os.chdir('/content')

!git clone https://github.com/jarredou/MVSEP-MDX23-Colab_v2.git &> /dev/null

os.chdir('/content/MVSEP-MDX23-Colab_v2')

!pip install -r requirements.txt &> /dev/null
# onnxruntime-gpu nightly fix for cuda12.2
!python -m pip install ort-nightly-gpu --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/
print('Installation done !')

Installing... This will take 1 minute...
Looking in indexes: https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/ort-cuda-12-nightly/pypi/simple/

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Installation done !


In [3]:
#@markdown #Separation
from pathlib import Path
import glob

os.chdir('/content/MVSEP-MDX23-Colab_v2')

input = '/content/dataset' #@param {type:"string"}
output_folder = '/content/output' #@param {type:"string"}
#@markdown ---
#@markdown *Bigshifts=1 to disable that feature*

BigShifts = 7 #@param {type:"slider", min:1, max:41, step:1}
#@markdown ---
overlap_InstVoc = 1 #@param {type:"slider", min:1, max:40, step:1}
overlap_VitLarge = 1 #@param {type:"slider", min:1, max:40, step:1}
#@markdown ---
weight_InstVoc = 8 #@param {type:"slider", min:0, max:10, step:1}
weight_VitLarge = 5 #@param {type:"slider", min:0, max:10, step:1}
#@markdown ---
use_VOCFT = False #@param {type:"boolean"}
overlap_VOCFT = 0.1 #@param {type:"slider", min:0, max:0.95, step:0.05}
weight_VOCFT = 2 #@param {type:"slider", min:0, max:10, step:1}
#@markdown ---
vocals_instru_only = True #@param {type:"boolean"}
overlap_demucs = 0.6 #@param {type:"slider", min:0, max:0.95, step:0.05}
#@markdown ---
output_format = 'PCM_16' #@param ["PCM_16", "FLOAT"]
if vocals_instru_only:
    vocals_only = '--vocals_only true'
else:
    vocals_only = ''


if use_VOCFT:
    use_VOCFT = '--use_VOCFT true'
else:
    use_VOCFT = ''

if Path(input).is_file():
  file_path = input
  Path(output_folder).mkdir(parents=True, exist_ok=True)
  !python inference.py \
        --large_gpu \
        --weight_InstVoc {weight_InstVoc} \
        --weight_VOCFT {weight_VOCFT} \
        --weight_VitLarge {weight_VitLarge} \
        --input_audio "{file_path}" \
        --overlap_demucs {overlap_demucs} \
        --overlap_VOCFT {overlap_VOCFT} \
        --overlap_InstVoc {overlap_InstVoc} \
        --overlap_VitLarge {overlap_VitLarge} \
        --output_format {output_format} \
        --BigShifts {BigShifts} \
        --output_folder "{output_folder}" \
        {vocals_only} \
        {use_VOCFT}

else:
  file_paths = sorted([f'"{glob.escape(path)}"' for path in glob.glob(input + "/*")])[:]
  input_audio_args = ' '.join(file_paths)
  Path(output_folder).mkdir(parents=True, exist_ok=True)
  !python inference.py \
          --large_gpu \
          --weight_InstVoc {weight_InstVoc} \
          --weight_VOCFT {weight_VOCFT} \
          --weight_VitLarge {weight_VitLarge} \
          --input_audio {input_audio_args} \
          --overlap_demucs {overlap_demucs} \
          --overlap_VOCFT {overlap_VOCFT} \
          --overlap_InstVoc {int(overlap_InstVoc)} \
          --overlap_VitLarge {int(overlap_VitLarge)} \
          --output_format {output_format} \
          --BigShifts {BigShifts} \
          --output_folder "{output_folder}" \
          {vocals_only} \
          {use_VOCFT}


GPU use: 0
started!

Options: 
BigShifts: 7

weight_InstVoc: 8.0
weight_VitLarge: 5.0

overlap_InstVoc: 1
overlap_VitLarge: 1

use_VOCFT: False
vocals_only: True
output_format: PCM_16

Loading InstVoc into memory
100%|████████████████████████████████████████| 427M/427M [00:54<00:00, 8.26MB/s]
100%|███████████████████████████████████████████| 709/709 [00:00<00:00, 811kB/s]
Loading VitLarge into memory
100%|████████████████████████████████████████| 824M/824M [01:36<00:00, 8.91MB/s]
100%|██████████████████████████████████████| 1.21k/1.21k [00:00<00:00, 12.6kB/s]
Go for: /content/dataset/Huh_Ori_HP-KAROKEE-MSB2-3BAND-3090_arch-124m_Vocals_converted.wav
Input audio: (2, 13159152) Sample rate: 44100
Processing vocals with VitLarge model...
 57%|█████████████████████████▋                   | 4/7 [00:36<00:33, 11.03s/it]^C


# Generate Dataset (Isolate Vocals)

## Setting Environment

In [6]:
#@markdown ### Input files
#@markdown Track name / any link (Upload your songs in tracks folder)
input = "https://www.youtube.com/watch?v=nSzwuS-mvkk&t=90s" #@param {type:"string"}
input = 'https://www.youtube.com/watch?v=6RQ-bBdASvk'

In [7]:
import os
os.chdir('/content')
# ------------VSC REWRITE
# pyright: reportMissingImports=false, reportUnusedVariable=warning, reportUntypedBaseClass=error
from IPython.display import Audio, display
#from google.colab import output
#from google.colab import drive
#from google.colab import files
from sys import exit
import zipfile
import hashlib
import os.path
import shutil
import psutil
import random
import glob
import time
import zlib
import sys
import torch 
from pathvalidate import sanitize_filename # for inference cell
import yt_dlp as youtube_dl # for inference cell

isCPU = torch.cuda.is_available()
#@markdown Uncheck if you want to use VocalRemover5 without mounting to drive.
MountDrive = False #@param{type:"boolean"}
#@markdown Use mounting method
method = 'new' #@param ["new","old"]
#@markdown Mounting path; don't touch this if you don't know what you're doing
mounting_path = '/content/drive/MyDrive' #@param ["snippets:","/content/drive/MyDrive","/content/drive/Shareddrives/<your shared drive name>", "/content/drive/Shareddrives/Shared Drive"]{allow-input: true}
#@markdown Force trigger update
ForceUpdate = False #@param{type:"boolean"}

#update channels
ai = 'https://github.com/NaJeongMo/Colaboratory-Notebook-for-Ultimate-Vocal-Remover'
vercheck = 'https://raw.githubusercontent.com/NaJeongMo/Colaboratory-Notebook-for-Ultimate-Vocal-Remover/main/v'
model_ver = 'https://raw.githubusercontent.com/NaJeongMo/Colaboratory-Notebook-for-Ultimate-Vocal-Remover/main/model_list'

class hide_opt: # hide outputs
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def get_size(bytes, suffix='B'): # read ram
    global svmem
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f'{bytes:.2f}{unit}{suffix}'
        bytes /= factor
    svmem = psutil.virtual_memory()

def validateModelLinks():
    with hide_opt():
        get_ipython().system('wget {} -O model_ver'.format(model_ver))
    model_ver_ = open("model_ver", "r")
    model_ver_ = model_ver_.read()
    with hide_opt():
        get_ipython().system('wget {} -O model_list'.format(model_ver_))
    model_list = open("model_list", "r")
    model_list = model_list.readlines()
    models = []
    for i in model_list:
        models.append(i)
    os.remove('model_ver')
    os.remove('model_list')
    return models

def installAI():
    print('Installing ai...', end=' ')
    get_ipython().system('git clone {} VocalRemover5-COLAB_arch'.format(ai))
    os.chdir('VocalRemover5-COLAB_arch')
    print('done')

    print('Downloading models...', end=' ')
    for i in validateModelLinks():
        with hide_opt():
            zname = sanitize_filename(os.path.basename(i))
            get_ipython().system("wget {}".format(i))
            get_ipython().system('unzip -o {}'.format(zname))
            os.remove(zname)
    print('done')

def dlvr(): # download vr to colab only
    print('Warning: changes will not be saved.')
    print('Downloading VR5...')

    os.chdir('/content')
    installAI()
    #os.chdir('/content/VocalRemover5-COLAB_arch')
    print ("Success!")

def check_update():
    if os.path.isdir(f'{mounting_path}/VocalRemover5-COLAB_arch'): # check update if ai installed is True
        os.chdir(f'{mounting_path}/VocalRemover5-COLAB_arch')
        print("Checking for updates...", end=" ")
        with hide_opt():
            get_ipython().system('wget {} -O check_ver'.format(vercheck))
        f = open("check_ver", "r")
        nver = f.read()
        f = open("v", "r")
        cver = f.read()
        if cver != nver or ForceUpdate:
            print('New update found! {}'.format(nver))
            choice = str(input('Do you want to update? y/n')).lower()
            if choice == 'y':
                os.chdir('../')
                print('Updating ai...',end=' ')
                with hide_opt():
                    get_ipython().system('git clone {} temp_VocalRemover5-COLAB_arch'.format(ai))
                get_ipython().system('cp -a temp_VocalRemover5-COLAB_arch/* VocalRemover5-COLAB_arch/')
                get_ipython().system('rm -rf temp_VocalRemover5-COLAB_arch')
                print('done')

                print('Downloading models...', end=' ')
                os.chdir('VocalRemover5-COLAB_arch')
                for i in validateModelLinks():
                    with hide_opt():
                        zname = sanitize_filename(os.path.basename(i))
                        get_ipython().system("wget {}".format(i))
                        get_ipython().system('unzip -o {}'.format(zname))
                        os.remove(zname)
                print('done')
                output.clear()
                os.remove(f'{mounting_path}/VocalRemover5-COLAB_arch/v')
                os.rename(f'{mounting_path}/VocalRemover5-COLAB_arch/check_ver',f'{mounting_path}/VocalRemover5-COLAB_arch/v')
                os.chdir(f'{mounting_path}/VocalRemover5-COLAB_arch') # just to make sure

            else:
                print('Skipping update.')
                os.remove(f'{mounting_path}/VocalRemover5-COLAB_arch/v')
                os.rename(f'{mounting_path}/VocalRemover5-COLAB_arch/check_ver',f'{mounting_path}/VocalRemover5-COLAB_arch/v')
                os.chdir(f'{mounting_path}/VocalRemover5-COLAB_arch') # just to make sure
        else:
            os.remove('check_ver')
            print('No update found.')
    else:
            if os.path.isdir('/content/VocalRemover5-COLAB_arch'):
                print("Success!")
            else:
                dlvr()

#-------------Script begin-------------
if os.path.isdir('/content/VocalRemover5-COLAB_arch') == False:
    if os.path.isdir('/content/VocalRemover5-COLAB_arch'):
        print('Success!')
    else:
        dlvr()


In [8]:

#----------DEFINE INFERENCE REQ------------
def crc32(fileName):
    with open(fileName, 'rb') as fh:
        hash = 0
        while True:
            s = fh.read(65536)
            if not s:
                break
            hash = zlib.crc32(s, hash)
        return "%08X" % (hash & 0xFFFFFFFF)
def YouTube(link, dl=True):
    inputsha = hashlib.sha1(bytes(link, encoding='utf8')).hexdigest() + '.wav'
    fmt = '251/140/250/139'
    opt = {'format': fmt, 'outtmpl': inputsha, 'updatetime': False, 'nocheckcertificate': True}
    if dl == True:
        print('YouTube link detected')
        print('Downloading...', end=' ')
    with hide_opt():
        with youtube_dl.YoutubeDL(opt) as ydl:
            global desc
            if dl == True:
                desc = ydl.extract_info(link, download=not os.path.isfile(inputsha))
            else:
                desc = ydl.extract_info(link, download=not True)

    titlename = sanitize_filename(desc['title'])
    if dl == True:
        print('done')
        print(titlename)
    if dl == True:
        return titlename, inputsha
    else:
        return titlename
def zipdir(folder, zipname): # LINUX CALL MODIFIED!!!
    if '.zip' in zipname:
        zipname = zipname.strip('.zip')
    get_ipython().system('zip -r {}.zip {}'.format(zipname,folder))
# dlFile(input,pretrained_model,isYouTube='http://' in input,export_as_mp3=export_as_mp3)
def dlFile(track,pretrained_model,isYouTube=False,export_as_mp3=False):
    modelname = os.path.splitext(os.path.basename(pretrained_model))[0]
    stems = [f'_{modelname}_Instruments',f'_{modelname}_Vocals']
    filename = os.path.splitext(os.path.basename(track))[0]
    if isYouTube:
        filename = YouTube(track,dl=False)
    if os.path.isdir(filename):
        shutil.rmtree(filename)
        os.mkdir(filename)
    else:
        os.mkdir(filename)
    if os.path.isfile(f'{filename}.zip'):
        os.remove(f'{filename}.zip')
    if export_as_mp3:
        os.chdir('separated/')
        for i in stems:
            wav_path = filename + i + '.wav'
            mp3_path = filename + i + '.mp3'
            get_ipython().system(f'ffmpeg -y -i "{wav_path}" -vn -ar 44100 -ac 2 -b:a 320k "{mp3_path}" -loglevel quiet')
        os.chdir('../')
        for move in stems:
            shutil.move('separated/' + filename + move + '.mp3',filename)
    else:
        for move in stems:
            shutil.move('separated/' + filename + move + '.wav',filename)
    with hide_opt():
        get_ipython().system(f'zip -r "{filename}.zip" "{filename}"')
    shutil.rmtree(filename)
    files.download(f'{filename}.zip')

In [9]:
# ------------VSC REWRITE
#type: ignore
if os.path.isfile('main.py') == False:
    if MountDrive:
        os.chdir(mounting_path + '/VocalRemover5-COLAB_arch')
    else:
        os.chdir('/content/VocalRemover5-COLAB_arch')

#parameter markdowns-----------------


ScanSeparatedFolder = False #@param {type:"boolean"}
#@markdown Convert all files in your tracks folder
convertAll = False #@param {type:"boolean"}
if convertAll:
    convertAll = '--convert_all'
else:
    convertAll = ''
#@markdown Model name (Upload your models in models folder)
pretrained_model = "HP2-4BAND-3090_4band_arch-500m_1.pth" #@param ["HighPrecison_4band_arch-124m_1.pth","HighPrecison_4band_arch-124m_2.pth","HP2-4BAND-3090_4band_arch-500m_1.pth","HP2-4BAND-3090_4band_arch-500m_2.pth","HP_4BAND_3090_arch-124m.pth","LOFI_2band-1_arch-34m.pth","LOFI_2band-2_arch-34m.pth","NewLayer_4band_arch-130m_1.pth","NewLayer_4band_arch-130m_2.pth","NewLayer_4band_arch-130m_3.pth","Vocal_HP_4BAND_3090_AGG_arch-124m.pth","Vocal_HP_4BAND_3090_arch-124m.pth","HP-KAROKEE-MSB2-3BAND-3090_arch-124m.pth","HP2-MAIN-MSB2-3BAND-3090_arch-500m.pth","HP-4BAND-V2_arch-124m.pth","MGM-v5-2Band-32000-_arch-default-BETA1.pth","MGM-v5-2Band-32000-_arch-default-BETA2.pth","MGM-v5-3Band-44100-_arch-default-BETA.pth","MGM-v5-4Band-44100-_arch-default-BETA1.pth","MGM-v5-4Band-44100-_arch-default-BETA2.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA1.pth","MGM-v5-KAROKEE-32000-_arch-default-BETA2-AGR.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA1.pth","MGM-v5-MIDSIDE-44100-_arch-default-BETA2.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA1.pth","MGM-v5-Vocal_2Band-32000-_arch-default-BETA2.pth","StackedMGM_1band_arch-default.pth"]{allow-input: true}
#@markdown ---
#@markdown ### Arguments
window_size =  512#@param {type:"integer"}
if window_size < 320:
    print('Warning: window_size lower than 320.')
if window_size < 272:
    window_size = 272
#1band_sr32000_hl512.json  2band_44100_lofi.json  3band_44100_mid.json
#1band_sr44100_hl512.json  2band_48000.json	 4band_44100.json
#2band_32000.json	  3band_44100.json	 ensemble.json
parameter = "Auto detect" #@param ["Auto detect","1band_sr32000_hl512.json","1band_sr44100_hl512.json", "2band_32000.json" , "2band_44100_lofi.json", "2band_48000.json", "3band_44100.json", "3band_44100_mid.json", "3band_44100_msb2.json", "4band_44100.json", "4band_v2.json"]
parameter = "modelparams/" + parameter
high_end_process = 'mirroring' #@param ["none","mirroring", "mirroring2" , "bypass"]
aggressiveness = '0.3' #@param {type:"string"}
aggressiveness = float(aggressiveness)
#@markdown Mute low volume vocals
postprocess = False #@param {type: "boolean"}
if postprocess:
    threshold = 0.2 #@param {type:"number"}
    min_range = 64 #?param {type:"integer"}
    fade_size = 32 #?param {type:"integer"}
    if min_range < fade_size * 2:
        print('min_range must be greater than or equal fade_size * 2')
        print('Using default instead. (except threshold)')
        min_range = 32
        fade_size = 64
    postprocess = f"-p -thres {threshold} -mrange {min_range} -fsize {fade_size}"
else:
    postprocess = ""
#@markdown ### Architecture
nn_architecture = 'Auto detect' #@param ["Auto detect","default", "34 MB", "124 MB", "130 MB", "500 MB"]
if nn_architecture == '34 MB':
    nn_architecture = '33966KB'
elif nn_architecture == '124 MB':
    nn_architecture = '123821KB'
elif nn_architecture == '130 MB':
    nn_architecture = '129605KB'
elif nn_architecture == '500 MB':
    nn_architecture = '537238KB'
elif nn_architecture == 'default':
    pass
#@markdown ---
#@markdown ### Checkboxes
#@markdown Use GPU for faster conversion
gpu = True #@param {type: "boolean"}
if gpu == True:
    gpu = 0
else:
    gpu = -1
#@markdown Aggressively remove vocals from Instrumental
deepExtraction = False #@param {type:"boolean"}
if deepExtraction:
    deepExtraction = "-D"
else:
    deepExtraction = ""
#@markdown Flip Instruments and Vocals output (Only for Vocal Models)
isVocal = False #@param {type:"boolean"}
if isVocal:
    isVocal = '--isVocal'
else:
    isVocal = ''
#@markdown Hide warnings
suppress = True #@param {type:"boolean"}
if suppress:
    suppress = '--suppress'
else:
    suppress = ''
#@markdown Export spectogram image
output_image = False #@param {type: "boolean"}
if output_image:
    output_image = "-I"
else:
    output_image = ""
#@markdown perform Test Time Augmentation to improve the separation quality
tta = True #@param {type: "boolean"}
if tta:
    tta = "-t"
else:
    tta = ""
#@markdown Use custom arguments
useCustomArguments = False #@param {type: "boolean"}
CustomArguments = "-h" #@param {type:"string"}
#@markdown Download files
download = False #@param {type:"boolean"}
export_as_mp3 = False #@param {type:"boolean"}
#@markdown Use all model
model_version = "Don't use all model" #@param ["Don't use all model","v5_new", "v5", "all"]
# {none,v5,v5_new,all}
# automation
if nn_architecture == 'Auto detect':
    if 'arch-default' in pretrained_model:
        nn_architecture = 'default'
    elif 'arch-34m' in pretrained_model:
        nn_architecture = '33966KB'
    elif 'arch-124m' in pretrained_model:
        nn_architecture = '123821KB'
    elif 'arch-130m' in pretrained_model:
        nn_architecture = '129605KB'
    elif 'arch-500m' in pretrained_model:
        nn_architecture = '537238KB'
    else:
        print('Error! autoDetect_arch')
        print('Using 124 MB instead.')
        nn_architecture = '123821KB'
if parameter == 'modelparams/Auto detect':
    if '4band' in pretrained_model.lower():
        if 'v2' in pretrained_model.lower():
            parameter = 'modelparams/4band_v2.json'
        else:
            parameter = 'modelparams/4band_44100.json'
    elif '3band' in pretrained_model.lower():
        if 'msb2' in pretrained_model.lower():
            parameter = 'modelparams/3band_44100_msb2.json'
        else:
            parameter = 'modelparams/3band_44100.json'
    elif 'midside' in pretrained_model.lower():
        parameter = 'modelparams/3band_44100_mid.json'
    elif '2band' in pretrained_model.lower():
        if 'lofi' in pretrained_model.lower():
            parameter = 'modelparams/2band_44100_lofi.json'
        else:
            parameter = 'modelparams/2band_48000.json'
    else:
        print('Parameter auto detect failed, using 1band instead.')
        parameter = 'modelparams/1band_sr44100_hl512.json'
if '34m' in pretrained_model or '124m' in pretrained_model or '130m' in pretrained_model or '500m' in pretrained_model:
    pretrained_model = "models/v5_new/" + pretrained_model
else:
    pretrained_model = "models/v5/" + pretrained_model
    if os.path.isfile(pretrained_model) == False:
        print('========================================================')
        print('                Error model not found.')
        print('Custom models should be uploaded in "models/v5/" folder!')
        print('========================================================')
if 'https://' not in input:
    if ScanSeparatedFolder:
        if input in ''.join(glob.glob('separated/*')):
            input = 'separated/' + input
        else:
            print('File not found in separated folder.')
            input = 'tracks/' + input
    else:
        input = 'tracks/' + input
elif 'https://' in input and ScanSeparatedFolder:
    print('Skipping "Separated" folder scan since a link is given.')

if model_version == "Don't use all model":
    model_version = 'none'

# --------------AI----------------
start_time = time.time()
#window_size,os.path.splitext(os.path.basename(pretrained_model)[0]),os.path.splitext(os.path.basename(parameter)[0],settings_tta = 'True' if tta else 'False'
#settings_tta = 'True' if tta else 'False',os.path.splitext(os.path.basename(parameter)[0],os.path.splitext(os.path.basename(pretrained_model)[0]),window_size
settings_tta = 'True' if tta else 'False'
settings_deepExtraction = 'True' if deepExtraction else 'False'

print('Window size: {}'.format(window_size))
print('Model: {}'.format(os.path.splitext(os.path.basename(pretrained_model))[0]))
print('Parameter: {}'.format(os.path.splitext(os.path.basename(parameter))[0]))
print('Aggressiveness: {}'.format(aggressiveness))
print('High end process: {}'.format(high_end_process))

print('TTA: {}'.format(settings_tta))
print('Deep Extraction: {}'.format(settings_deepExtraction))
print()
if useCustomArguments == False:
    get_ipython().system(f'python3.8 main.py -i "{input}" {convertAll} --useAllModel "{model_version}" --model_params "{parameter}" -P "{pretrained_model}" -w {window_size} -H "{high_end_process}" --aggressiveness {aggressiveness} -n "{nn_architecture}" -g {gpu} {deepExtraction} {isVocal} {suppress} {output_image} {postprocess} {tta}')
    if download and convertAll:
        sys.exit("No no, this is not an error but downloading with convertAll is not yet possible. Please DON'T report this to me (Hv) or the server")
    if download:
        dlFile(input,pretrained_model,isYouTube='https://' in input,export_as_mp3=export_as_mp3)
if useCustomArguments:
    get_ipython().system(f'python3.8 main.py {CustomArguments}')
print('Notebook took: {0:.{1}f}s'.format(time.time() - start_time, 1))

Window size: 512
Model: HP2-4BAND-3090_4band_arch-500m_1
Parameter: 4band_44100
Aggressiveness: 0.3
High end process: mirroring
TTA: True
Deep Extraction: False

YouTube Link detected
Downloading... done
성시경(Sungsikyung)의 킬링보이스를 라이브로! - 너의 모든 순간, 좋을텐데, 넌 감동이었어, 희재, 두사람, 미소천사, 거리에서, 태양계, I Love Uㅣ딩고뮤직
loading model... done
stft of wave source... done
100%|█████████████████████████████████████████| 449/449 [02:14<00:00,  3.34it/s]
inverse stft of Instruments... done
inverse stft of Vocals... done
Total time: 410.9s
Notebook took: 414.6s


# Voice Model Training

## Dependencies

In [ ]:
#@title Clone Repositories
import os

firsttry = True

# READ ME BEFORE CHANGING THINGS
# If you're attempting to replace the imports here with Applio-RVC, it will not work due to requirement discrepancies across the entire notebook.
# I will not be porting this notebook to Applio due to the failure of the Applio team to provide backwards compatibility with the Crepe and Mangio-Crepe f0 feature format.
# DO NOT ASK. IT WILL NOT HAPPEN.

os.chdir('/content/')

#if(os.path.exists("/content/Mangio-RVC-Fork")):
#  print("RVC already installed, skipping.")
#else:
#  #Credit to miaaaa0a on the AI Hub Discord for (indirectly) suggesting this variant of Mangio RVC to me.
#  !git clone -b pr-optimization --single-branch https://github.com/alexlnkp/Mangio-RVC-Tweaks.git
#  #Rename to keep backwards compatibility with old variants of Disconnected
#  os.rename("/content/Mangio-RVC-Tweaks", "/content/Mangio-RVC-Fork")
#  !git clone https://github.com/maxrmorrison/torchcrepe.git
#  !mv torchcrepe/torchcrepe Mangio-RVC-Fork/
#  !rm -rf torchcrepe  # Delete the torchcrepe repository folder

os.chdir('/content/Mangio-RVC-Fork')
now_dir = "/content/Mangio-RVC-Fork"
os.makedirs(os.path.join(now_dir, "logs"), exist_ok=True)
os.makedirs(os.path.join(now_dir, "weights"), exist_ok=True)

In [ ]:
#@title GPU Check
import torch

ngpu = torch.cuda.device_count()
gpu_infos = []
mem = []
if_gpu_ok = False

if torch.cuda.is_available() or ngpu != 0:
  for i in range(ngpu):
    gpu_name = torch.cuda.get_device_name(i)
    if any(
        value in gpu_name.upper()
        for value in ["10", "16", "20", "30", "40", "A2", "A3", "A4", "P4", "A50", "500", "A60", "70", "80", "90", "M4", "T4", "TITAN"]
    ):
      if_gpu_ok = True
      print("Compatible GPU detected: %s" % gpu_name)
      gpu_infos.append("%s\t%s" % (i, gpu_name))
      mem.append(int(torch.cuda.get_device_properties(i).total_memory / 1024 / 1024 / 1024 + 0.4))

if if_gpu_ok and len(gpu_infos) > 0:
  gpu_info = "\n".join(gpu_infos)

else:
  raise Exception("No GPU detected; training cannot continue. Please change your runtime type to a GPU.")
gpus = "-".join(i[0] for i in gpu_infos)


In [ ]:
#@title Mount Drive
os.makedirs('/content/rvcDisconnected', exist_ok=True)

In [ ]:
#@title Setup CSVDB
#...Alright, you made your point.
import csv

if not os.path.isdir("csvdb/"):
  os.makedirs("csvdb")
  frmnt, stp = open("csvdb/formanting.csv", "w", newline=""), open("csvdb/stop.csv", "w", newline="")
  csv_writer = csv.writer(frmnt, delimiter=",")
  csv_writer.writerow([False, 1.0, 1.0])
  csv_writer = csv.writer(stp, delimiter=",")
  csv_writer.writerow([False])
  frmnt.close()
  stp.close()

global DoFormant, Quefrency, Timbre
DoFormant, Quefrency, Timbre = False, 1.0, 1.0